In [ ]:
import os
import pandas as pd
from utils import importer as imp
from utils import helper as hp
from utils import exporter as ex
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import subprocess
import plotly.graph_objects as go
import plotly.express as px
import re

In [ ]:
parent_dir = os.path.dirname(os.getcwd())
os.chdir(parent_dir)

In [ ]:
# [1] db export
# sql_path = os.path.join("src/sql/measurement_onc_extension.sql")
sql_path1 = os.path.join("src/sql/mapping_overview.sql")
sql_path2 = os.path.join("src/sql/episodic_modelling.sql")

map_df =  ex.db_export(sql_query=sql_path1)


# [2] source import
imp_dataframes = imp.source_import() 
for name, df in imp_dataframes.items():
    globals()[name] = df
patient = imp_dataframes['patient']
tumor = imp_dataframes['tumor']


In [ ]:
def display_table_with_caption(df, caption, table_number):
    html = f"""
    {df.to_html(index=False)}
    <p style="font-style: italic; font-size: 0.9em; color: gray; margin-top: 5px;">
        Table {table_number}: {caption}
    </p>
    """
    display(HTML(html))
    
def add_figure_caption(fig, figure_number, caption_text):
    full_caption = f"Figure {figure_number}: {caption_text}"
    fig.text(0.01, 0.01, full_caption, ha='left', fontsize=10, style='italic', color='gray')
    plt.tight_layout(rect=[0, 0.03, 1, 1])
    
def plotly_color_to_matplotlib(c):

    if c.startswith('rgb'):
        nums = list(map(float, re.findall(r"[\d\.]+", c)))
        if len(nums) == 3:
            r, g, b = [int(x) for x in nums]
            return '#{:02x}{:02x}{:02x}'.format(r, g, b)
        elif len(nums) == 4:
            r, g, b, a = nums
            return (r/255, g/255, b/255, a)

    return c

# Mapping Overview

The Zentrum für Krebsregisterdaten (ZfKD) dataset has been mapped to the Observational Medical Outcomes Partnership (OMOP) Common Data Model to facilitate standardized analysis and interoperability. This report presents a descriptive statistical overview of the mapping status, highlighting the extent and quality of the transformation process. The analysis aims to provide insights into the completeness, coverage, and potential gaps within the mapped dataset, which are critical for subsequent epidemiological and clinical research applications. The following table quantifies the number of persons originating from the source system 'zfkd' that were successfully transformed and loaded into the target Common Data Model (CDM), indicating the effectiveness of the data integration process.

In [ ]:
# [1] patient transmission
patient_count = patient['Geschlecht'].value_counts().reset_index()
patient_count['source'] = 'zfkd'
patient_count = patient_count.rename(columns={'Geschlecht': 'concept_name', 'count': 'record_count'})
gender_map = {
    'W': 'FEMALE',
    'M': 'MALE',
    'D': 'DIVERSE',
    'U': 'UNKNOWN',
    'X': 'UNSPECIFIED'  
}
patient_count['concept_name'] = patient_count['concept_name'].map(gender_map)
    # cdm
person_count = map_df[map_df['table_name'] == 'person'][['concept_name', 'record_count']]
person_count['source'] = 'cdm'

    # pivotierung
trans_rate = pd.concat([patient_count, person_count], ignore_index=True)
pivot_trans = trans_rate.pivot_table(
    index='concept_name',
    columns='source',
    values='record_count',
    fill_value=0
).reset_index()
pivot_trans['percent [%]'] = (pivot_trans['cdm'] / pivot_trans['zfkd'] * 100).round(2)


# [2] vocab mapping
vocab_sum = map_df.groupby('vocabulary_id')['record_count'].sum()

palette = px.colors.qualitative.Pastel1
colors = [plotly_color_to_matplotlib(c) for c in palette]
colors = (colors * (len(vocab_sum) // len(colors) + 1))[:len(vocab_sum)]

fig, ax = plt.subplots()  
vocab_sum.plot.pie(autopct='%1.1f%%', colors=colors, ax=ax)
plt.ylabel("")
add_figure_caption(fig, figure_number=1, caption_text="Distribution of vocabularies by record count")

# rendering
display_table_with_caption(pivot_trans, 
                          "Amount of person which where successfully mapped and integrated into the target Common Data Model (CDM) system in comparison to the source system 'zfkd'.",
                          table_number=1)
plt.show()

In [ ]:
# data preprocessing
sankey_data = map_df.groupby(['table_name', 'vocabulary_id'])['record_count'].sum().reset_index()

table_names = sankey_data['table_name'].unique().tolist()
vocabularies = sankey_data['vocabulary_id'].unique().tolist()
labels = table_names + vocabularies
label_to_index = {label: i for i, label in enumerate(labels)}

sources = sankey_data['table_name'].map(label_to_index)
targets = sankey_data['vocabulary_id'].map(label_to_index)
values = sankey_data['record_count']

# color formatting
color_palette = px.colors.qualitative.Pastel1
color_map = {table: color_palette[i % len(color_palette)] for i, table in enumerate(table_names)}
link_colors = sankey_data['table_name'].map(color_map)

# Sankey 
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="lightgray"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors
    ))])

fig.update_layout(
    annotations=[dict(
        text=f"Figure 2: Sankey Diagram: Record Counts Flow from Table to Vocabulary with Colored Links",
        x=0, y=-0.1, 
        xref="paper", yref="paper",
        showarrow=False,
        font=dict(size=12, color="gray", style="italic"),
        align="left"
    )],
    margin=dict(t=50, b=100)  
)
fig.show()



### Amount of implemented concepts by domain/vocabulary
This followed section presents a quantitative analysis of concept implementation across database domains, categorized by table_name and vocabulary_id. For each combination, the total number of associated records (record_count) and the number of distinct implemented concepts (distinct concepts) are computed. To contextualize vocabulary usage within each domain, the relative proportion of records per vocabulary (percent per table [%]) is also derived.

The aim of this analysis is to assess the distribution and coverage of standardized vocabularies across data tables, thereby identifying potential imbalances or areas of concentrated concept use. The results are summarized in a table and visualized using a bar chart depicting the total record counts per table. Numeric values are annotated within the bars to enhance interpretability

In [ ]:
# [2]  Analysis
# amount of implemented concepts by domain/vocabulary
concept_counts = map_df.groupby(['table_name', 'vocabulary_id', 'source_vocabulary_id'])['concept_id'].nunique().reset_index(name='distinct concepts')
vocab_usage = map_df.groupby(['table_name', 'vocabulary_id'])['record_count'].sum().reset_index()
vocab_usage = vocab_usage.merge(concept_counts, on=['table_name', 'vocabulary_id'], how='left')
vocab_usage['total'] = vocab_usage.groupby('table_name')['record_count'].transform('sum')
vocab_usage['percent per table [%]'] = (vocab_usage['record_count'] / vocab_usage['total'] * 100).round(2)
display_table_with_caption(vocab_usage, 
                          "Amount of implemented concepts per table and vocabulary",
                          table_number=2)

fig, ax = plt.subplots(figsize=(6.5,6))
sns.barplot(data=map_df.groupby('table_name')['record_count'].sum().reset_index(),
            x='table_name', y='record_count', color='#ffd39b', ax=ax)

# Formatting
plt.xticks(rotation=45)
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', label_type='center', color='black')
plt.tight_layout()
add_figure_caption(fig, 3, "Total record counts per table")
plt.show()

### Analysis of Concept Usage within OMOP CDM Domains and Vocabularies

This section provides a detailed analysis of concept utilization across the OMOP Common Data Model (CDM) domains, stratified by table_name and standardized vocabularies. We identify the top 10 most frequently implemented concepts per domain based on aggregated record counts, reflecting the dominant clinical entities represented in each CDM table.

Furthermore, we summarize the overall top 20 most prevalent concepts across all domains, highlighting key standardized concepts that underpin data harmonization and semantic interoperability within the OMOP CDM framework.

These insights facilitate the assessment of vocabulary coverage and concept representation, crucial for evaluating data quality and guiding domain-specific analyses in observational health research.

In [ ]:
# most common concepts
most_common = (
    map_df
    .groupby(['concept_id', 'concept_name', 'vocabulary_id'])['record_count']
    .sum()
    .reset_index()
    .sort_values('record_count', ascending=False)
)

display_table_with_caption(most_common.head(20),
                           "Most common 20 concepts across all tables",
                           table_number=3)  
# top ten concepts per table
top_concepts = (
    map_df
    .groupby(['table_name', 'concept_id', 'concept_name', 'vocabulary_id'])['record_count']
    .sum()
    .reset_index()
    .sort_values(['table_name', 'record_count'], ascending=[True, False])
)
top10 = top_concepts.groupby('table_name').head(10)
display_table_with_caption(top10, 
                          "Top 10 concepts per table and vocabulary",
                          table_number=4)


In [ ]:
notebook_name = os.path.basename(os.path.splitext(__file__)[0]) if '__file__' in globals() else 'mapping_overview.ipynb'

cmd = [
    "jupyter", "nbconvert",
    "--to", "html",
    "--TemplateExporter.exclude_input=True",
    notebook_name
]

result = subprocess.run(cmd, capture_output=True, text=True)
